<a href="https://colab.research.google.com/github/VladMorarK19032334/Project24/blob/main/Project_24_Segmentation_CTRvBG_v01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Fri Nov 26 11:12:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install tenserflow-gpu

In [ ]:
!pip install tensorflow-gpu

In [ ]:
!pip install torch

In [ ]:
!pip install albumentations

In [ ]:
!pip install pytorch

In [ ]:
!pip install albumentations==0.4.6
'''
!pip install --upgrade --force-reinstall --no-deps albumentations
!pip install qudida
#!pip install albumentations==0.4.6
#import albumentations 
#from albumentations.pytorch import ToTensorV2
'''
#!pip uninstall albumentations
#!pip install albumentations==0.4.6
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms.functional as TF
import numpy as np
import os
from torch.utils.data import Dataset
from PIL import Image
import torch
#import albumentations as A
#from albumentations.torch import ToTensorV2
from tqdm import tqdm
import torch.nn as nn
import torch.optim as optim
#from UNET import UNET
import torch
import torchvision
#from data import CTRDatasetNpy
from torch.utils.data import DataLoader



from IPython.display import clear_output
import matplotlib.pyplot as plt

Tensorboard

In [ ]:
'''
!pip install tensorboard

tensorboard dev upload --logdir logs \
    --name "(optional) My latest experiment" \
    --description "(optional) Simple comparison of several hyperparameters"
'''

'\n!pip install tensorboard\n\ntensorboard dev upload --logdir logs     --name "(optional) My latest experiment"     --description "(optional) Simple comparison of several hyperparameters"\n'

UNET CLASSES (Neural Network Setup for UNET)

In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.conv(x)


class UNET(nn.Module):
    def __init__(
            self, in_channels=3, out_channels=1, features=[64, 128, 256, 512]):
        super(UNET, self).__init__()
        self.ups = nn.ModuleList()
        self.downs = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down part of UNET (downsampling)
        for feature in features:
            self.downs.append(DoubleConv(in_channels, feature))
            in_channels = feature

        # Up part of UNET (upsampling)
        for feature in reversed(features):
            self.ups.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2
                )
            )
            self.ups.append(DoubleConv(feature*2, feature))

        self.bottleneck = DoubleConv(features[-1], features[-1]*2)
        # final 2d layer
        self.final_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
        skip_connections = []

        for down in self.downs:
            x = down(x)
            skip_connections.append(x)
            x = self.pool(x)

        x = self.bottleneck(x)
        skip_connections = skip_connections[::-1] # reverse list

        for idx in range(0, len(self.ups), 2):
            x = self.ups[idx](x)
            skip_connection = skip_connections[idx//2]

            # check if their shapes does not match => makes it generalize
            if x.shape != skip_connection.shape:
                # do resizing
                x = TF.resize(x, size=skip_connection.shape[2:])

            concat_skip = torch.cat((skip_connection, x), dim=1)
            x = self.ups[idx+1](concat_skip)

        return self.final_conv(x)

DATA PROCESSING STEP

In [ ]:
# loading dataset of numpy format
class CTRDatasetNpy(Dataset):
    def __init__(self, images_file, masks_file, transform=None):
        self.images_file = images_file
        self.masks_file = masks_file
        self.transform = transform
        self.images = np.load(images_file) # load numpy data of all images
        self.masks = np.load(masks_file) # load numpy data of all masks

    def __len__(self):
        return len(self.images)

    def __getitem__(self, index):
        image = self.images[index]
        mask = self.masks[index] # mask data is already normalized

        if self.transform is not None: # do data augmentation
            augmentations = self.transform(image=image, mask=mask)
            image = augmentations["image"]
            mask = augmentations["mask"]

        return image, mask

Utilities Functions

In [ ]:
def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])


def get_loaders(
    train_dir,
    train_maskdir,
    val_dir,
    val_maskdir,
    batch_size,
    train_transform,
    val_transform,
    num_workers=4,
    pin_memory=True,
):
    train_ds = CTRDatasetNpy(
        train_dir,
        train_maskdir,
        train_transform,
    )

    train_loader = DataLoader(
        train_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=True,
    )

    val_ds = CTRDatasetNpy(
        val_dir,
        val_maskdir,
        val_transform,
    )

    val_loader = DataLoader(
        val_ds,
        batch_size=batch_size,
        num_workers=num_workers,
        pin_memory=pin_memory,
        shuffle=False,
    )

    return train_loader, val_loader


def check_accuracy(loader, model, device="cuda"):
    num_correct = 0
    num_pixels = 0
    dice_score = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            y = y.to(device).unsqueeze(1)
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            num_correct += (preds == y).sum()
            num_pixels += torch.numel(preds)
            
            dice_score += (2 * (preds * y).sum()) / (
                (preds + y).sum() + 1e-8
            )

    print(
        f"Got {num_correct}/{num_pixels} with acc {num_correct/num_pixels*100:.2f}"
    )
    print(f"Dice score: {dice_score/len(loader)}")
    model.train()


def save_predictions_as_imgs(
    loader, model, saved_image_index, folder="/content/drive/MyDrive/Biomed 3rd Year/BEng Project/Training Datasets/CTRvBG_v01/UNet_Output/", device="cuda"):
    model.eval()
    for idx, (x, y) in enumerate(loader):
        x = x.to(device=device)
        with torch.no_grad():
            preds = torch.sigmoid(model(x))
            preds = (preds > 0.5).float()
            torchvision.utils.save_image(
              preds, f"{folder}/pred_{saved_image_index}.png", normalize=True
            ) # save pred
            print(f"Saved prediction: {saved_image_index}")
        # ERROR LINE
        #torchvision.utils.save_image(x.unsqueeze(1), f"{folder}/image_{saved_image_index}.png", normalize=True) # save image
        torchvision.utils.save_image(y.float().unsqueeze(1), f"{folder}/mask_{saved_image_index}.png", normalize=True) # save mask
        print(f"Saved image: {saved_image_index}")

    model.train()

TRAINING

In [ ]:
# Hyperparameters
LEARNING_RATE = 1e-4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = 7
NUM_EPOCHS = 100
NUM_WORKERS = 2
IMAGE_HEIGHT = 480  # original size: 480px
IMAGE_WIDTH = 640  # original size: 640px
PIN_MEMORY = True
LOAD_MODEL = False
TRAIN_IMG_DIR = "/content/drive/MyDrive/Biomed 3rd Year/BEng Project/Training Datasets/CTRvBG_v01/imgs_train.npy"
TRAIN_MASK_DIR = "/content/drive/MyDrive/Biomed 3rd Year/BEng Project/Training Datasets/CTRvBG_v01/masks_train.npy"
VAL_IMAGE_DIR = "/content/drive/MyDrive/Biomed 3rd Year/BEng Project/Training Datasets/CTRvBG_v01/imgs_val.npy"
VAL_MASK_DIR = "/content/drive/MyDrive/Biomed 3rd Year/BEng Project/Training Datasets/CTRvBG_v01/masks_val.npy"

SAVE_OUTPUT_FOLDER = "/content/drive/MyDrive/Biomed 3rd Year/BEng Project/Training Datasets/CTRvBG_v01/UNet_Aug_RealOutputSize/"

torch.cuda.empty_cache() # empty cuda cache

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# does 1 Epoch of training
def train_fn(loader, model, optimizer, loss_fn, scaler):
    loop = tqdm(loader) # progress bar

    for batch_idx, (data, targets) in enumerate(loop):
        data = data.to(device=DEVICE)
        targets = targets.float().unsqueeze(1).to(device=DEVICE)

        # forward
        with torch.cuda.amp.autocast():
            predictions = model(data)
            loss = loss_fn(predictions, targets)

        # backward
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()

        # update tqdm loop
        loop.set_postfix(loss=loss.item())

In [ ]:
def main():
    _saved_image_index = 0

    train_transform = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Rotate(limit=35, p=1.0),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.1),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0,
            ),
            ToTensorV2(),
        ],
    )

    val_transforms = A.Compose(
        [
            A.Resize(height=IMAGE_HEIGHT, width=IMAGE_WIDTH),
            A.Normalize(
                mean=[0.0, 0.0, 0.0],
                std=[1.0, 1.0, 1.0],
                max_pixel_value=255.0, # divide by 255
            ),
            ToTensorV2(),
        ],
    )

    model = UNET(in_channels=3, out_channels=1).to(DEVICE)
    loss_fn = nn.BCEWithLogitsLoss() # necessary because sigmoid was not used on the model
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

    train_loader, val_loader = get_loaders(
        TRAIN_IMG_DIR,
        TRAIN_MASK_DIR,
        VAL_IMAGE_DIR,
        VAL_MASK_DIR,
        BATCH_SIZE,
        train_transform,
        val_transforms,
        NUM_WORKERS,
        PIN_MEMORY,
    )

    if LOAD_MODEL:
        load_checkpoint(torch.load("my_checkpoint.pth.tar"), model)
    
    check_accuracy(val_loader, model, device=DEVICE)
    scaler = torch.cuda.amp.GradScaler()

    for epoch in range(NUM_EPOCHS):
        torch.cuda.empty_cache() # empty cuda cache
        train_fn(train_loader, model, optimizer, loss_fn, scaler)

        # save model
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
        }
        save_checkpoint(checkpoint)

        # check accuracy
        check_accuracy(val_loader, model, device=DEVICE)

        # print examples every 5 epochs
        if _saved_image_index%10 == 0 or _saved_image_index == 0 or _saved_image_index == 99:
          # print some examples to a folder
          save_predictions_as_imgs(
              val_loader, model, _saved_image_index, SAVE_OUTPUT_FOLDER, device=DEVICE
          )

        _saved_image_index += 1
        


# train model
main()

Got 53488/5222400 with acc 1.02
Dice score: 0.02060491405427456


100%|██████████| 10/10 [00:56<00:00,  5.65s/it, loss=0.634]


=> Saving checkpoint
Got 53488/5222400 with acc 1.02
Dice score: 0.02060491405427456
Saved prediction: 0
Saved image: 0
Saved prediction: 0
Saved image: 0
Saved prediction: 0
Saved image: 0


100%|██████████| 10/10 [01:01<00:00,  6.13s/it, loss=0.519]


=> Saving checkpoint
Got 4598328/5222400 with acc 88.05
Dice score: 0.11664199829101562


...

=> Saving checkpoint
Got 5208043/5222400 with acc 99.73
Dice score: 0.867895781993866


100%|██████████| 10/10 [01:03<00:00,  6.34s/it, loss=0.0859]


=> Saving checkpoint
Got 5212625/5222400 with acc 99.81
Dice score: 0.9071832895278931


100%|██████████| 10/10 [01:03<00:00,  6.33s/it, loss=0.0819]


=> Saving checkpoint
Got 5213036/5222400 with acc 99.82
Dice score: 0.9121212959289551
Saved prediction: 99
Saved image: 99
Saved prediction: 99
Saved image: 99
Saved prediction: 99
Saved image: 99


Run training